![Nvidia Logo](./images/nvidia.png)  

## 5G Network Configuration Agent

### Overview  
This notebook outlines how the 5G network configuration works—how it detects SDU buffer full errors and reconfigures the network. We will use concepts demonstrated in [intro_agents.ipynb](intro_agents.ipynb) to build this agent using LangGraph and LangChain.

### Table of Contents
1. Architecture Overview
2. File Descriptions
3. Define and run the Agent
4. Streamlit UI implementation

### 1. Architecture Overview

![Architecture diagram](./images/architecture_diagram.png)  

#### Key Components:  

**Agents**:
1. **Monitoring Agent**:  
    - Continuously reads gNodeB logs from `../llm-slicing-5g-lab/logs/gNodeB.log`.  
    - Analyzes each chunk for SDU buffer full errors.  
    - Routes to the Configuration Agent if an error is detected.  

2. **Configuration Agent**:  
    - Called when an error is detected in the gNodeB logs.  
    - Has two tools bound to it: `get_packet_loss` and `reconfigure_network`.  
    - First, retrieves the latest packet loss logs from the database using the `get_packet_loss` tool.  
    - Analyzes the logs and determines which UE needs more bandwidth. Based on this, it assigns higher bandwidth to the selected UE.  
    - Calls the `reconfigure_network` tool to use xAPP and reconfigure the network.  
    - Returns control to the Monitoring Agent to continue monitoring.  

**Tools**:
1. **`get_packet_loss`**: Queries the database and retrieves a DataFrame containing per-UE packet loss statistics.  
2. **`reconfigure_network`**: Calls the xAPP with optimized slicing parameters to adjust network configurations.  

#### Example Error Logs  

```md
[RLC]   /home/nvidia/llm-slicing-5g-lab/openairinterface5g/openair2/LAYER2/nr_rlc/nr_rlc_entity_am.c:1769:nr_rlc_entity_am_recv_sdu: warning: 195 SDU rejected, SDU buffer full
                [NR_MAC]   Frame.Slot 896.0
                UE RNTI c1f9 CU-UE-ID 1 in-sync PH 0 dB PCMAX 0 dBm, average RSRP -44 (16 meas)
                UE c1f9: UL-RI 1, TPMI 0
                UE c1f9: dlsch_rounds 23415/1/0/0, dlsch_errors 0, pucch0_DTX 0, BLER 0.00000 MCS (0) 28
                UE c1f9: ulsch_rounds 8560/0/0/0, ulsch_errors 0, ulsch_DTX 0, BLER 0.00000 MCS (0) 9
                UE c1f9: MAC:    TX      177738642 RX         612401 bytes
                UE c1f9: LCID 1: TX           1022 RX            325 bytes
```
  
### 2. Files to Refer  

- **[agents.py](./agents.py)** – Contains code for Monitoring and Configuration Agents.  
- **[tools.py](./tools.py)** – Implements the tools used by the agents.  
- **[langgraph_agent.py](./langgraph_agent.py)** – Defines the LangGraph agent workflow.  
- **[chatbot_DLI.py](./chatbot_DLI.py)** – Implementation for the Streamlit UI.  


#### Expected Output  

By the end of this notebook, you will have:  
- A functional LangGraph workflow connected to the 5g slicing lab, that detects network issues and triggers reconfiguration.  
- A pipeline capable of analyzing logs, querying packet loss data, and adjusting slicing parameters dynamically.  

### Creating a LangGraph Workflow  

We have defined two agents—the **Monitoring Agent** and the **Configuration Agent**—as combinations of a model and the tool(s) they have access to. This is achieved using LangGraph's `create_react_agent()` function, which creates an agent that employs ReAct prompting.

**States in Graph**  
        - A state represents the evolving context of execution, maintaining data across multiple steps.  
        - It stores intermediate results, tool outputs, and agent decisions.  
        - States enable reasoning over past interactions, ensuring continuity in the workflow.  

**Nodes and Edges in LangGraph**  
        - **Nodes** represent agents, tool calls, or decision steps in the workflow.  
        - **Edges** define the flow between nodes, determining execution order based on conditions.  
        - This structure allows dynamic decision-making and parallel execution where needed.  

Refer [this](https://langchain-ai.github.io/langgraph/concepts/low_level/) for more information.

The workflow has been defined in [langgraph_agent.py](langgraph_agent.py), please refer it for implementation details. 

### Running the Streamlit User Interface

We provide a predefined Streamlit-based user interface for monitoring the system in real time. This interface allows users to interact with the monitoring software efficiently and gain insights into its operation.

#### About Streamlit:
[Streamlit](https://streamlit.io/) is a lightweight Python framework for building interactive web applications with minimal effort. It enables users to create and deploy data-driven dashboards and tools using simple Python scripts.

#### Features of the UI:
- Real-time Log Monitoring – View live logs generated by the agent.
- Packet Loss Visualization – Monitor real-time packet loss of UE1 and UE2 using dynamic charts.
- Agent Control – Start and stop the agent directly through the UI.

In [1]:
%%js
var url = 'http://'+window.location.host+'/dashboard';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Streamlit Frontend ></h2></a>';

<IPython.core.display.Javascript object>

In [ ]:
!~/.local/bin/streamlit run chatbot_DLI.py --server.enableCORS=false --server.enableXsrfProtection=false




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.27.19.176:8501
  External URL: http://204.52.26.81:8501



### Running Langgraph Agent

The streamlit UI calls langgraph_agent.py in the background. The agent logs its outputs to agent.log, which are in turn displayed on the UI. You may run the script to see how the agent works. Log files are written to in the `/llm-slicing-5g-lab/logs` directory. Run the following commands in separate terminals to stream logs for agent, UE1 and UE2 respectively.

```sh
tail -f /llm-slicing-5g-lab/logs/agent.log
tail -f /llm-slicing-5g-lab/logs/UE2_iperfc.log
tail -f /llm-slicing-5g-lab/logs/UE1_iperfc.log
```



In [ ]:
!python3 langgraph_agent.py